In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelBinarizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading Data
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [3]:
# Creating validation data and scaling data to range (0-1)
X_valid, X_train = X_train[:4000], X_train[4000:] / 255.0
y_valid, y_train = y_train[:4000], y_train[4000:]
X_test = X_test / 255.0

In [4]:
# Using a smaller subset for quicker experimentation
X_train_small = X_train[:10000]
y_train_small = y_train[:10000]
X_valid_small = X_valid[:1000]
y_valid_small = y_valid[:1000]

In [5]:
# Reshape data to include channel dimension
X_train_small = X_train_small.reshape(-1, 28, 28, 1)
X_valid_small = X_valid_small.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [6]:
# Function to build and compile CNN model
def build_compile_cnn_model(learning_rate):
    model = keras.models.Sequential([
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(70, activation='relu'),
        keras.layers.Dense(50, activation='relu'),
        keras.layers.Dense(20, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model


In [ ]:
# Experiment with different learning rates
learning_rates = [0.1, 0.01, 0.001, 0.0001]
results = []

for lr in learning_rates:
    print(f'\nTraining CNN with learning rate: {lr}')
    cnn_model = build_compile_cnn_model(lr)
    cnn_model_history = cnn_model.fit(X_train_small, y_train_small, validation_data=(X_valid_small, y_valid_small), epochs=10, verbose=0)
    
    # Evaluate model
    test_loss, test_acc = cnn_model.evaluate(X_test, y_test, verbose=0)
    
    # Make predictions
    y_probs = cnn_model.predict(X_test)
    y_preds = y_probs.argmax(axis=1)
    
    # Compute metrics
    label_binarizer = LabelBinarizer()
    y_test_binary = label_binarizer.fit_transform(y_test)
    y_test_categorical = np.argmax(y_test_binary, axis=1)
    y_pred_categorical = np.argmax(y_probs, axis=1)
    
    accuracy = accuracy_score(y_test_categorical, y_pred_categorical)
    precision = precision_score(y_test_categorical, y_pred_categorical, average='weighted')
    recall = recall_score(y_test_categorical, y_pred_categorical, average='weighted')
    f1 = f1_score(y_test_categorical, y_pred_categorical, average='weighted')
    
    results.append({
        'learning_rate': lr,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'train_loss': cnn_model_history.history['loss'],
        'val_loss': cnn_model_history.history['val_loss'],
        'train_accuracy': cnn_model_history.history['accuracy'],
        'val_accuracy': cnn_model_history.history['val_accuracy']
    })


Training CNN with learning rate: 0.1
313/313 [==============================] - 1s 3ms/step

Training CNN with learning rate: 0.01
313/313 [==============================] - 1s 3ms/step

Training CNN with learning rate: 0.001


In [ ]:
# Plotting the results
epochs = range(1, 11)

for result in results:
    lr = result['learning_rate']
    
    # Plotting Training and Validation Loss
    plt.figure(figsize=(12, 6))
    plt.style.use('fivethirtyeight')
    sns.lineplot(epochs, result['train_loss'], label=f'Train Loss (lr={lr})')
    sns.lineplot(epochs, result['val_loss'], label=f'Val Loss (lr={lr})')
    plt.title(f'Training and Validation Loss for Learning Rate {lr}\n')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(loc='best')
    plt.show()
    
    # Plotting Training and Validation Accuracy
    plt.figure(figsize=(12, 6))
    sns.lineplot(epochs, result['train_accuracy'], label=f'Train Accuracy (lr={lr})')
    sns.lineplot(epochs, result['val_accuracy'], label=f'Val Accuracy (lr={lr})')
    plt.title(f'Training and Validation Accuracy for Learning Rate {lr}\n')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')
    plt.show()


In [ ]:
# Print final metrics for each learning rate
for result in results:
    print(f"Learning Rate: {result['learning_rate']}")
    print(f"  Accuracy: {result['accuracy']}")
    print(f"  Precision: {result['precision']}")
    print(f"  Recall: {result['recall']}")
    print(f"  F1 Score: {result['f1']}")
    print()